In [91]:
def interp_coeffs(xvals, yvals):
    """
    Return the divided-difference interpolating polynomial
    coefficients based on xvals and yvals. The degree of
    the interpolating polynomial will be inferred from data,
    which will be 1 less than the number of data points.
    """
    assert len(xvals)==len(yvals)
    nbr_data_points = len(xvals)

    # sort inputs by xvals =>
    data = sorted(zip(xvals, yvals), reverse=False)
    xvals, yvals = zip(*data)

    depth      = 1
    coeffs     = [yvals[0]]
    iter_yvals = yvals

    while depth < nbr_data_points:

        iterdata = []

        for i in range(len(iter_yvals)-1):

            delta_y = iter_yvals[i+1]-iter_yvals[i]
            delta_x = xvals[i+depth]-xvals[i]
            iterval = (delta_y/delta_x)
            iterdata.append(iterval)

            # append top-most entries from table to coeffs =>
            if i==0: coeffs.append(iterval)

        iter_yvals = iterdata
        depth+=1

    return(coeffs)




def interp_val(xvals, yvals):
    """
    Return a function representing the interpolating
    polynomial determined by xvals and yvals.
    """
    assert len(xvals)==len(yvals)
    nbr_data_points = len(xvals)

    # sort inputs by xvals =>
    data = sorted(zip(xvals, yvals), reverse=False)
    xvals, yvals = zip(*data)

    depth      = 1
    coeffs     = [yvals[0]]
    iter_yvals = yvals

    while depth < nbr_data_points:

        iterdata = []

        for i in range(len(iter_yvals)-1):

            delta_y = iter_yvals[i+1]-iter_yvals[i]
            delta_x = xvals[i+depth]-xvals[i]
            iterval = (delta_y/delta_x)
            iterdata.append(iterval)

            # append top-most entries in tree to coeffs =>
            if i==0: coeffs.append(iterval)

        iter_yvals = iterdata
        depth+=1

    def f(i):
        """
        Evaluate interpolated estimate at x.
        """
        terms  = []
        retval = 0

        for j in range(len(coeffs)):

            iterval   = coeffs[j]
            iterxvals = xvals[:j]
            for k in iterxvals: iterval*=(i-k)
            terms.append(iterval)
            retval+=iterval
        return(retval)
    return(f)

In [129]:
from scipy.interpolate import lagrange
import numpy as np
from sympy import Symbol
import sympy as sp
X = Symbol('x')
x = np.array([0, 1, 2, 3])
y = np.array([1, 1, 7, 25])
L = {}
for i in range(len(x)):
    L[f'L{i}'] = 1
    for xi in x:
        if xi != x[i]:
            L[f'L{i}'] *= (X - xi)/(x[i] - xi)

for key in L:
    print(f'{key}: {sp.expand(L[key])}')

L0: -x**3/6 + x**2 - 11*x/6 + 1
L1: x**3/2 - 5*x**2/2 + 3*x
L2: -x**3/2 + 2*x**2 - 3*x/2
L3: x**3/6 - x**2/2 + x/3


In [130]:
pt = 1.5
for key in L:
    print(f'{key}({pt}) = {L[key].subs(X, pt)}')

L0(1.5) = -0.0625000000000000
L1(1.5) = 0.562500000000000
L2(1.5) = 0.562500000000000
L3(1.5) = -0.0625000000000000


In [ ]:
poly = lagrange(x, y)
poly

In [106]:
x = np.array([0, 0.3, 0.5, 0.75, 1])
y = np.array([1, 2, 3, 2, 1])
divs = interp_coeffs(x, y)
divs

[1,
 3.3333333333333335,
 3.333333333333333,
 -31.11111111111111,
 59.682539682539684]

In [107]:
px = divs[0]
for i, coef in enumerate(divs[1:]):
    mult = 1
    for j in range(i+1):
        mult *= (X - x[j])
    px += coef*mult
px.simplify()

59.6825396825397*x**4 - 123.619047619048*x**3 + 72.984126984127*x**2 - 9.04761904761905*x + 1.0

In [99]:
f_fin = interp_val(x, y)
f_fin(0.25)

1.7916666666666667

In [110]:
f = sp.cos(X/4)
aprox = 1.5
# x = np.array([1, 1.1, 1.2])
# y = np.exp(x)
n = 2
f_der = f
for _ in range(n+1):
    f_der = sp.diff(f_der)
f_der

exp(x)

In [111]:
maj = f_der.subs(X, 1.2)
err = 1
for xi in x:
    err *= (X - xi)
err = sp.Abs(err) * maj/sp.factorial(n+1)
err.subs(X, aprox)

0.000207507307671034